# Análise exploratória dos Dados

## Sobre o Dataset

Este é um dataset sobre pedidos em ecommerce brasileiro realizado pela Olist Store. O dataset contém informações sobre 100 mil pedidos feitos em diversas lojas durante o período de 2016 a 2018. O dataset permite visualizar os pedidos a partir de múltiplas dimensões. De status do pedido, preço, pagamento e performance do frete até a localização do cliente, atributos do produto e, finalmente, avaliações escritas pelos clientes. Também há um conjunto de dados de geolocalização que relaciona os códigos postais brasileiros às coordenadas lat/long.

São dados comerciais reais, foram anonimizados e as referências às empresas e parceiros no texto da revisão foram substituídas pelos nomes das grandes casas de Game of Thrones.


## Atenção
1. Um pedido pode ter vários itens.
2. Cada item pode ser atendido por um vendedor distinto.
3. Todo o texto identificando lojas e parceiros foi substituído pelos nomes das grandes casas de Game of Thrones.

## Data Schema
Os dados são divididos em vários conjuntos de dados para melhor compreensão e organização. Consulte o esquema de dados a seguir ao trabalhar com ele:

<img src="assets/data_schema.png" width="1000px" />



**Customers Dataset (dataset de clientes)**<br/><br/>
Este conjunto de dados contém informações sobre o cliente e sua localização. Use-o para identificar clientes únicos no conjunto de dados de pedidos e para encontrar o local de entrega dos pedidos.

No sistema, cada pedido é atribuído a um customerid exclusivo. Isso significa que o mesmo cliente receberá IDs diferentes para pedidos diferentes. O objetivo de ter um customerunique_id no conjunto de dados é permitir que você identifique clientes que fizeram recompras na loja. Caso contrário, você descobriria que cada pedido tinha um cliente diferente associado.

---

**Geolocation Dataset (Conjunto de dados de geolocalização)**<br/><br/>
Este conjunto de dados contém informações sobre códigos postais brasileiros e suas coordenadas lat/lng. Use-o para traçar mapas e encontrar distâncias entre vendedores e clientes.

---

**Order Items Dataset (Conjunto de dados de itens de pedido)**<br/><br/>
Este conjunto de dados inclui dados sobre os itens comprados em cada pedido.

---

**Payments Dataset (Conjunto de dados de pagamentos)**<br/><br/>
Este conjunto de dados inclui dados sobre as opções de pagamento dos pedidos.

---

**Order Reviews Dataset (Conjunto de dados de avaliações de pedidos)**<br/><br/>
Este conjunto de dados inclui dados sobre as avaliações feitas pelos clientes.

Depois que um cliente compra o produto da Olist Store, um vendedor é notificado para atender a esse pedido. Assim que o cliente recebe o produto, ou a data prevista de entrega está prevista, o cliente recebe um inquérito de satisfação por e-mail onde pode dar nota da experiência de compra e deixar alguns comentários.

---

**Order Dataset (Conjunto de dados do pedido)**<br/><br/>
Este é o conjunto de dados principal. De cada pedido você pode encontrar todas as outras informações.

---

**Products Dataset (Conjunto de dados de produtos)**<br/><br/>
Este conjunto de dados inclui dados sobre os produtos vendidos pela Olist.

---

**Sellers Dataset (Conjunto de dados de vendedores)**<br/><br/>
Este conjunto de dados inclui dados sobre os vendedores que atenderam aos pedidos feitos na Olist. Use-o para encontrar a localização do vendedor e identificar qual vendedor entregou cada produto.

---

**Tradução do nome da categoria**<br/><br/>
Traduz o nome da categoria do produto para o inglês.